# Entanglement Capabilities of Different Ansatzes

In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from analysis_functions import Analysis
import sys
sys.path.append("../")
from vqc.vqc_circuits import UQC

To measure entanglement capability of the different ansatzes, we are going to use the Meyer-Wallach entanglement measure $Q$ of the states generated by it. Here, there are actually three different things we can do:
 - 1) Simply characterize the entanglement capability of the ansatz by measuring Q of the PQC with random weights.
 - 2) Measure Q of the PQC with the final weights and see how much entanglement the PQC has at the end of training.
 - 3) Measure Q every x training steps and see how entanglement evolves during training.

Measure the entanglement of a state:

$$Q(\ket{\psi}) = 2\left(1 - \frac{1}{n}\sum_{j=1}^{n}Tr(\rho_j^2)\right)$$

Measure the entanglement cabaibility of a PQC:

$$Q(\ket{\psi}) = \frac{2}{|S|}\sum_{\theta_i\in S}\left(1 - \frac{1}{n}\sum_{j=1}^{n}Tr(\rho_j^2(\theta_i))\right)$$

## 1) Entanglement Capability of the different ansatzes

In [16]:
def skolik_variational_layer(wires, params):
    [qml.RY(params[i,0], wires[i]) for i in range(len(wires))]
    [qml.RZ(params[i,1], wires[i]) for i in range(len(wires))]

def skolik_entangling_layer(wires):
    [qml.CZ(wires = [i,j]) for i,j in zip(wires, wires[1:])]
    if len(wires) != 2:
        qml.CZ(wires = [wires[0], wires[-1]])

def cnots_circular_layer(wires):
    [qml.CNOT(wires = [i,j]) for i,j in zip(wires, wires[1:])]
    if len(wires) != 2:
        qml.CNOT(wires = [wires[0], wires[-1]])

def full_entangling_layer_cnot(wires):
    [qml.CNOT(wires = [i,j]) for i in wires for j in wires if i != j]

def full_entangling_layer_cz(wires):
    [qml.CZ(wires = [i,j]) for i in wires for j in wires if i != j]

def skolik_data_encoding(wires, data, params):
    [qml.RX(data[i] * params[i], wires[i], id = f"x_{i}") for i in range(len(wires))]

def uqc_layer( wires, data, rotational_weights, input_weights, bias_weights):
    [qml.RZ(np.dot(2 * input_weights[i], data) + bias_weights[i] , wires[i]) for i in range(len(wires))]
    [qml.RY(2 * rotational_weights[i], wires[i]) for i in range(len(wires))]

def schuld_datareup(params, num_qubits, num_layers, data, qubit_to_measure, entangling_layer):
    input_weights = params[0]
    rotational_weights = params[1]
    for l in range(num_layers):
        skolik_variational_layer(range(4), rotational_weights[l])
        entangling_layer(range(4))
        skolik_data_encoding(range(4), data, input_weights[l])
    skolik_variational_layer(range(4), rotational_weights[num_layers-1])
    return qml.density_matrix(qubit_to_measure)

def uqc(params, num_qubits, num_layers, data, qubit_to_measure, entangling_layer):
    rotational_weights = params[0]
    input_weights = params[1]
    bias_weights = params[2]
    for l in range(num_layers):
        uqc_layer(range(num_qubits), data, rotational_weights[l], input_weights[l], bias_weights[l])
        entangling_layer(range(num_qubits))
    return qml.density_matrix(qubit_to_measure)

In [17]:
dev_4qubits = qml.device("default.qubit", wires = 4)
dev_2qubits = qml.device("default.qubit", wires = 2)

skolik_datareup_circuit = qml.QNode(schuld_datareup, dev_4qubits)
uqc_2qubits_circuit = qml.QNode(uqc, dev_2qubits)
uqc_4qubits_circuit = qml.QNode(uqc, dev_4qubits)

In [18]:
def q(circuit,weights, num_qubits, num_layers, data, entangling_layer):
    """
    Returns the Meyer-Wallach measure of entanglement of the state produced by the circuit
    """
    entropy = 0
    for j in range(num_qubits):
        reduced_density_matrix = circuit(weights, num_qubits, num_layers, data, j, entangling_layer)
        trace = np.trace(np.matmul(reduced_density_matrix, reduced_density_matrix))
        entropy += trace
    entropy /= num_qubits
    entropy = 1 - entropy
    return 2*entropy

def entangling_capability(circuit, num_qubits, num_layers, circuit_arch, entangling_layer, sample = 1024):
    """
    Uses the Meyer-Wallach measure of entanglement to compute the entangling capability of the circuit
    """
    res = np.zeros(sample, dtype = complex)

    for i in range(sample):
        if circuit_arch == "uqc":
            params = [np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits)),
                      np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits, 4)),
                      np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits))]
        else:
            params = [np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits)),
                      np.random.uniform(low = 0, high = 2*np.pi, size = (num_layers, num_qubits, 2))]
            
        data = np.random.uniform(low = -0.05, high = 0.05, size = 4)
        res[i] = q(circuit, params, num_qubits, num_layers, data, entangling_layer)
    
    return np.sum(res).real/sample

In [19]:
entangling_capability_uqc_4qubits = entangling_capability(uqc_4qubits_circuit, 4, 5, "uqc", skolik_entangling_layer)
entangling_capability_uqc_2qubits = entangling_capability(uqc_2qubits_circuit, 2, 5, "uqc", skolik_entangling_layer)

In [21]:
entangling_capability_skolik_skolik_entangling = entangling_capability(skolik_datareup_circuit, 4, 5, "schuld", skolik_entangling_layer)
entangling_capability_skolik_cnots_circular_layer = entangling_capability(skolik_datareup_circuit, 4, 5, "schuld", cnots_circular_layer)
entangling_capability_skolik_full_entangling_layer_cnot = entangling_capability(skolik_datareup_circuit, 4, 5, "schuld", full_entangling_layer_cnot)
entangling_capability_skolik_full_entangling_layer_cz = entangling_capability(skolik_datareup_circuit, 4, 5, "schuld", full_entangling_layer_cz)

In [22]:
print("Entangling capability of UQC with 4 qubits: ", entangling_capability_uqc_4qubits)
print("Entangling capability of UQC with 2 qubits: ", entangling_capability_uqc_2qubits)
print("Entangling capability of Schuld with 4 qubits and Skolik entangling layer: ", entangling_capability_skolik_skolik_entangling)
print("Entangling capability of Schuld with 4 qubits and CNOTs circular entangling layer: ", entangling_capability_skolik_cnots_circular_layer)
print("Entangling capability of Schuld with 4 qubits and full entangling layer with CNOTs: ", entangling_capability_skolik_full_entangling_layer_cnot)
print("Entangling capability of Schuld with 4 qubits and full entangling layer with CZs: ", entangling_capability_skolik_full_entangling_layer_cz)

Entangling capability of UQC with 4 qubits:  0.7632545177443731
Entangling capability of UQC with 2 qubits:  0.40746327849309516
Entangling capability of Schuld with 4 qubits and Skolik entangling layer:  0.7659601272074706
Entangling capability of Schuld with 4 qubits and CNOTs circular entangling layer:  0.814661721196705
Entangling capability of Schuld with 4 qubits and full entangling layer with CNOTs:  0.8082769721485377
Entangling capability of Schuld with 4 qubits and full entangling layer with CZs:  -1.218122824830914e-14
